In [1]:
from dsx.ds_utils import *

Package loaded in Notebook Mode
Successfully imported ds_utils as Package


In [2]:
import re, datetime, qgrid, dtale
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('fivethirtyeight')
sns.set_context(context={'figure.figsize': (16,9)})

In [4]:
qgrid.set_grid_option('forceFitColumns', False)

In [5]:
dsx.set_dirs()

Set project directory to /mnt/d/DSAI_Workstation/0003_ePrimer/workspace.
Property "dir_%" enabled


# Loading Data

In [6]:
file_excel = pd.ExcelFile('data/inputs/eprimer_raw.xlsx')
file_excel.sheet_names

['Learning History', 'Feedback']

In [7]:
df = file_excel.parse(file_excel.sheet_names[0])
dfx = file_excel.parse(file_excel.sheet_names[1])

## One-off Agency Name and Short-Form

### Only Run If to Re-generate mapping file

In [8]:
df_temp = df[['Email', 'Agency']].drop_duplicates()
df_temp.ds.rename('Agency', 'Agency_Descr')

In [ ]:
df_short = pd.read_csv('data/inputs/Learning History.csv')
df_short = df_short[['Email', 'Agency']].drop_duplicates()

In [ ]:
df_mapps = df_temp.ds.merge(df_short, 'left', 'Email')
df_mapps = df_mapps[['Agency', 'Agency_Descr']].drop_duplicates('Agency_Descr')

df_mapps.ds.duplicated('Agency_Descr') == 0
df_mapps.dropna(axis=0, subset=['Agency'], inplace=True)

df_mapps = df_mapps[['Agency_Descr', 'Agency']]

In [ ]:
df_mapps.to_excel('data/inputs/mapps_agency.xlsx', index=False)

### Mapping Agency to Acronym 

In [9]:
df_mapps = pd.read_excel('data/inputs/mapps_agency.xlsx')

In [10]:
df.ds.rename('Agency', 'Agency_Descr')
dfx.ds.rename('Agency', 'Agency_Descr')

df = df.ds.merge(df_mapps, 'left', 'Agency_Descr', isnull='Agency')
dfx = dfx.ds.merge(df_mapps, 'left', 'Agency_Descr', isnull='Agency')

(True, 24392)
(71, 0.0029107904230895377)
(True, 10271)
(21, 0.0020445915684938177)


## Cleaning

In [11]:
# Standardized Field Names
df.ds.stdcols()
dfx.ds.stdcols()

In [12]:
df = df.ds.merge(dfx[['Email', 'CourseID', 'Rated_Date_Time_SGT', 'Rating', 'Comment']], 'left', ['Email', 'CourseID'], isnull='Rated_Date_Time_SGT')

(True, 24392)
(14121, 0.5789193178091178)


In [13]:
df['Module'] = df.Courses_Name.str.extract(r'([0-9]{1})')
df['Module_Descr'] = 'M' + df.Module
df.Module = df.Module.astype(int)

In [14]:
df['Module_Complete'] = df.Lesson_Completion == 100

In [15]:
df.loc[(df.Course_Completed_Date_SGT.isnull()) & (df.Module_Complete), 'Course_Completed_Date_SGT'] = df.Lesson_Completed_Date_SGT

In [16]:
cols_drop = ['Courses_Name', 'Course_Domain', 'Tag', 'Time_Spent', 'Lesson_Name', 'Lesson_Completion', 'Lesson_Completed_Date_SGT']
df.drop(cols_drop, 1, inplace=True)

In [17]:
df.dropna(axis=0, subset=['Agency'], inplace=True)

# Calculated Features

## Day of the Week

In [18]:
df['enrolled_dow'] = df.Course_Enrolled_Date_SGT.map(lambda x: x.weekday())
df['completed_dow'] = df.Course_Completed_Date_SGT.map(lambda x: x.weekday())

In [19]:
mapps_dow = {
    0 : 'Sun',
    1 : 'Mon',
    2 : 'Tues',
    3 : 'Wed',
    4 : 'Thurs',
    5 : 'Fri',
    6 : 'Sat'
}

df['enrolled_dow_label'] = df.enrolled_dow.map(mapps_dow)
df['completed_dow_label'] = df.completed_dow.map(mapps_dow)

## Time Spent

In [20]:
dt_delta = df.Course_Completed_Date_SGT - df.Course_Enrolled_Date_SGT
df['total_minutes'] = dt_delta.map(lambda x: x.total_seconds() // 60)
df['total_days'] = df.total_minutes / 60 / 24

## Enrolled & Completed Module(s) Prior to Current Module

In [21]:
def funx(rw):
    dff = df[df.Email == rw['Email']].copy()
    dff = dff[dff.Course_Enrolled_Date_SGT != rw['Course_Enrolled_Date_SGT']]
    return len(dff[dff.Course_Enrolled_Date_SGT < rw['Course_Enrolled_Date_SGT']])

In [22]:
df['enrol_prior_module_count'] = df.parallel_apply(funx, axis=1)

## Sequence of Modules

## Number of People that Enrolled in Same Module on Same Day

In [24]:
gp_enrol_avg_per_day = df[df.Module == 1].groupby(pd.Grouper(key=df.Course_Enrolled_Date_SGT.name, freq='7d')).Email.agg(lambda x: len(x)/7)
gp_enrol_avg_per_day.index.rename('enrol_date', inplace=True)

In [25]:
gp_enrol_avg_per_day.name = 'enrol_average_d7'

In [26]:
df.set_index('Course_Enrolled_Date_SGT', drop=False, inplace=True)
df.index.rename('enrol_date', inplace=True)

In [27]:
df.sort_index(0, inplace=True)
df = pd.merge_asof(df,  gp_enrol_avg_per_day, left_on='enrol_date', right_on='enrol_date', direction='backward')

# Export Data

In [28]:
df.to_excel('data/outputs/processed_data.xlsx', index=False, encoding='utf-8')